Name : Saarthak Khamkar   
Roll No. : D088   
SAP ID : 60009230057

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as T
from torchvision.datasets import CIFAR10
import torchvision.models as models
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler  # mixed precision

# Data Augmentation

In [ ]:
simclr_transform = T.Compose([
    T.RandomResizedCrop(32),
    T.RandomHorizontalFlip(),
    T.RandomApply([T.ColorJitter(0.4,0.4,0.4,0.1)], p=0.8),
    T.RandomGrayscale(p=0.2),
    T.GaussianBlur(kernel_size=3),
    T.ToTensor(),
    T.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261])
])

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=simclr_transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)

100%|██████████| 170M/170M [00:10<00:00, 16.1MB/s]
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


# Encoder Network

In [ ]:
class Encoder(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.backbone = nn.Sequential(*list(base_model.children())[:-1])
        self.fc = nn.Linear(base_model.fc.in_features, 128)

    def forward(self, x):
        x = self.backbone(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return F.normalize(x, dim=1)


# Projection Head

In [ ]:
class ProjectionHead(nn.Module):
    def __init__(self, in_dim=128, out_dim=128):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, 512)
        self.fc2 = nn.Linear(512, out_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# SimCLR Model

In [ ]:
class SimCLR(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.encoder = Encoder(base_model)
        self.projection_head = ProjectionHead()

    def forward(self, x):
        x = self.encoder(x)
        x = self.projection_head(x)
        return x

# NT-Xent Loss

In [ ]:
class NTXentLoss(nn.Module):
    def __init__(self, temperature=0.5):
        super().__init__()
        self.temperature = temperature

    def forward(self, z_i, z_j):
        z = torch.cat([z_i, z_j], dim=0)  # 2N x D
        sim = F.cosine_similarity(z.unsqueeze(1), z.unsqueeze(0), dim=2) / self.temperature
        mask = torch.eye(sim.size(0), device=z.device).bool()
        sim = sim.masked_fill(mask, -9e15)  # mask self-similarity
        labels = torch.arange(z_i.size(0), device=z.device)
        labels = torch.cat([labels, labels], dim=0)
        loss = F.cross_entropy(sim, labels)
        return loss

# Training Loop

In [ ]:
def train(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    scaler = GradScaler()  # mixed precision

    for x, _ in tqdm(loader):
        x = x.to(device)
        # Create two views for contrastive learning
        x_i = x
        x_j = x.clone()  # You can also apply a separate augmentation here

        optimizer.zero_grad()
        with autocast():  # mixed precision
            z_i, z_j = model(x_i), model(x_j)
            loss = criterion(z_i, z_j)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
    return total_loss / len(loader)

# Main

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model = models.resnet18(pretrained=True)  # use pretrained for faster convergence
model = SimCLR(base_model).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = NTXentLoss().to(device)

for epoch in range(1):
    loss = train(model, train_loader, optimizer, criterion, device)
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}")

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 167MB/s]
/tmp/ipython-input-973147274.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # mixed precision
/usr/local/lib/python3.12/dist-packages/torch/amp/grad_scaler.py:136: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
  0%|          | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/tmp/ipython-input-973147274.py:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # mixed precision
/usr/local/lib/python3.12/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_ty

Epoch 1, Loss: 4500000175654456.5000


Colab Link : https://colab.research.google.com/drive/13bivmbUU0z0gGcpg-vnuLDBTfEdAEtkj?usp=sharing